In [122]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim


In [123]:
data = pd.read_csv("C:/Users/29943/Desktop/模式识别/实验数据集/Iris数据集/iris.csv")
data.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [124]:
label = data['Species'].unique()
label

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [125]:
#选取每个类别的前30个样本作为训练集，后20个样本作为测试集
train_data = pd.DataFrame()
test_data = pd.DataFrame()
for i in label:
    train_data = train_data.append(data[data['Species']==i][:30])
    test_data = test_data.append(data[data['Species']==i][30:])

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

train_data.head()
test_data.head()

C:\Users\29943\AppData\Local\Temp\ipykernel_6088\769161672.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(data[data['Species']==i][:30])
C:\Users\29943\AppData\Local\Temp\ipykernel_6088\769161672.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(data[data['Species']==i][30:])
C:\Users\29943\AppData\Local\Temp\ipykernel_6088\769161672.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(data[data['Species']==i][:30])
C:\Users\29943\AppData\Local\Temp\ipykernel_6088\769161672.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,31,4.8,3.1,1.6,0.2,setosa
1,32,5.4,3.4,1.5,0.4,setosa
2,33,5.2,4.1,1.5,0.1,setosa
3,34,5.5,4.2,1.4,0.2,setosa
4,35,4.9,3.1,1.5,0.2,setosa


In [126]:
#将数据转换为dataloder
from torch.utils.data import Dataset, DataLoader
class IrisDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.x = torch.tensor(data.iloc[:, 0:4].values, dtype=torch.float32)
        self.y = torch.tensor(data.iloc[:, 4].values, dtype=torch.long)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.data)
    
train_dataset = IrisDataset(train_data)
test_dataset = IrisDataset(test_data)

train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=True)

#打印train_loader的标签
for x, y in train_loader:
    print(y)
    break

tensor([1, 1, 1, 1, 1, 2, 0, 2, 1, 0, 2, 1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
        1, 2, 0, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 1, 1, 1, 2, 1, 2, 2, 0, 0, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1,
        1, 2, 2, 2, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0])


In [127]:
def Softmax(x):
    return torch.exp(x) / torch.sum(torch.exp(x), dim=1).view(-1, 1)

In [128]:
#搭建神经网络，输入层为4个神经元，输出层为3个神经元，有四个隐层，第一层为10个神经元，第二层为20个神经元，第三层为10个神经元，第四层为5个神经元,第一层的激活函数为relu，第二层的激活函数为tanh，第三层的激活函数为sigmoid，第四层的激活函数为relu，第五层的激活函数为softmax
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4,10)
        self.fc2 = nn.Linear(10,20)
        self.fc3 = nn.Linear(20,10)
        self.fc4 = nn.Linear(10,5)
        self.fc5 = nn.Linear(5,3)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = Softmax(self.fc5(x))
        return x


In [129]:

def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = torch.argmax(y_hat, dim=1)
    cmp = y_hat.type(y.dtype) == y
    return float(torch.sum(cmp.type(y.dtype)))

def evaluate_accuracy(data_loader, net):
    acc_sum, n = 0.0, 0
    for x, y in data_loader:
        pred = net(x)
        acc_sum += accuracy(pred, y)
        n += 1
    return acc_sum / n

In [130]:
#训练模型,epoch_num为训练轮数,打印每轮训练的loss,训练集准确率,测试集准确率
def train(train_loader,test_loader, model, criterion, optimizer, epoch_num):
    model.train()
    for epoch in range(epoch_num):
        train_loss = 0.0
        train_acc = 0.0
        test_acc = 0.0
        for x, y in train_loader:
            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_acc += accuracy(pred, y)
        test_acc = evaluate_accuracy(test_loader, model)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f' % (epoch + 1, train_loss / len(train_loader), train_acc / len(train_dataset), test_acc/len(test_dataset)))




In [131]:
#开始训练
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train(train_loader, test_loader ,model, criterion, optimizer, 5000)


epoch 1, loss 1.0684, train acc 0.478, test acc 0.467
epoch 2, loss 1.0683, train acc 0.478, test acc 0.467
epoch 3, loss 1.0681, train acc 0.478, test acc 0.467
epoch 4, loss 1.0678, train acc 0.478, test acc 0.467
epoch 5, loss 1.0674, train acc 0.478, test acc 0.467
epoch 6, loss 1.0670, train acc 0.478, test acc 0.467
epoch 7, loss 1.0665, train acc 0.478, test acc 0.467
epoch 8, loss 1.0661, train acc 0.478, test acc 0.467
epoch 9, loss 1.0655, train acc 0.478, test acc 0.467
epoch 10, loss 1.0650, train acc 0.478, test acc 0.467
epoch 11, loss 1.0645, train acc 0.478, test acc 0.467
epoch 12, loss 1.0641, train acc 0.478, test acc 0.467
epoch 13, loss 1.0638, train acc 0.478, test acc 0.467
epoch 14, loss 1.0635, train acc 0.478, test acc 0.467
epoch 15, loss 1.0631, train acc 0.478, test acc 0.467
epoch 16, loss 1.0628, train acc 0.478, test acc 0.467
epoch 17, loss 1.0624, train acc 0.478, test acc 0.467
epoch 18, loss 1.0620, train acc 0.478, test acc 0.467
epoch 19, loss 1.06

epoch 36, loss 1.0551, train acc 0.478, test acc 0.467
epoch 37, loss 1.0548, train acc 0.478, test acc 0.467
epoch 38, loss 1.0544, train acc 0.478, test acc 0.467
epoch 39, loss 1.0540, train acc 0.478, test acc 0.467
epoch 40, loss 1.0536, train acc 0.478, test acc 0.467
epoch 41, loss 1.0533, train acc 0.478, test acc 0.467
epoch 42, loss 1.0529, train acc 0.478, test acc 0.467
epoch 43, loss 1.0526, train acc 0.478, test acc 0.467
epoch 44, loss 1.0523, train acc 0.478, test acc 0.467
epoch 45, loss 1.0520, train acc 0.478, test acc 0.467
epoch 46, loss 1.0517, train acc 0.478, test acc 0.467
epoch 47, loss 1.0514, train acc 0.478, test acc 0.467
epoch 48, loss 1.0511, train acc 0.478, test acc 0.467
epoch 49, loss 1.0508, train acc 0.478, test acc 0.467
epoch 50, loss 1.0506, train acc 0.478, test acc 0.467
epoch 51, loss 1.0503, train acc 0.478, test acc 0.467
epoch 52, loss 1.0501, train acc 0.478, test acc 0.467
epoch 53, loss 1.0498, train acc 0.478, test acc 0.467
epoch 54, 

In [132]:
#测试，打印真实标签和预测标签，统计预测正确的个数以及准确率
for x, y in test_loader:
    pred = model(x)
    pred = torch.argmax(pred, dim=1)
    print('pred: ', pred)
    print('true: ', y)
    print('acc: ', accuracy(pred, y) / len(y))
    break

    

pred:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1])
true:  tensor([2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 2, 0, 0, 2, 0, 0, 0,
        1, 0, 1, 0, 2, 1, 1, 1, 2, 1, 0, 1, 0, 1, 0, 0, 1, 0, 2, 2, 1, 1, 1, 0,
        1, 1, 0, 2, 1, 2, 1, 0, 1, 1, 0, 2])
acc:  0.5833333333333334
